<p align="center" ><img src="https://www.ai4kids.ai/wp-content/uploads/2019/07/ai4kids_website_logo_120x40.png"></img></p>

# 學AI真簡單 <1> 初探機器學習
## 第三章 3-4 傑克與蘿絲誰的生存機率高—決策樹分類器介紹與應用



<p align="right">© Copyright AI4kids.ai</p>

## 首先運行下面的程序，下載train.csv

In [ ]:
import os
if not os.path.exists('train.csv'):
    !wget --no-check-certificate "https://drive.google.com/uc?export=download&id=13bGRvk1Vq9tFRzMWsXZwOg7TzZLQj83O" -O 'train.csv'

### 資料清理

In [ ]:
from sklearn import tree
import numpy as np
import pandas as pd

data = pd.read_csv('train.csv')
data = data.drop(columns=['Name', 'Ticket', 'Cabin'])
data


### 資料轉換

In [ ]:
# 轉換Embarked欄位
typeEmbarked = list(set(data['Embarked']))

# 把Embarked欄位的文字按順序轉換成0~n的數字
for i in range(len(typeEmbarked)):
    print(typeEmbarked[i])
    row =  data['Embarked'] == typeEmbarked[i]
    data.loc[row, 'Embarked'] = i

# 轉換Sex欄位
typeSex = list(set(data['Sex']))

# 把Sex欄位的文字按順序轉換成0~n的數字
for i in range(len(typeSex)):
    print(typeSex[i])
    rows =  data['Sex'] == typeSex[i]
    data.loc[rows, 'Sex'] = i
    
data

鐵達尼資料集 - 進行分析1

Step 1.資料集中，取出最後有存活的資料集 data_sur

In [ ]:
data_sur = data[data['Survived']==1]

Step 2. 存活且為男性的資料集male，存活且為女性的資料集female

In [ ]:
df_male = data_sur[data_sur['Sex']==0]
df_female = data_sur[data_sur['Sex']==1]

Step 3. pandas內建的 plot.bar() 畫出柱狀圖來分析

In [ ]:
df = pd.DataFrame({'Sex':['Male', 'Female'], 'val':[len(df_male), len(df_female)]})
ax = df.plot.bar(x='Sex', y='val', rot=0)

鐵達尼資料集 - 進行分析2

那麼年輕人和中年人的存活比率是如何呢？
先取出資料集內存活並且小於30歲的資料集 df_young, 和存活並大於30歲的資料集 df_older

In [ ]:
df_young = data_sur[data_sur['Age'] < 30]
df_older = data_sur[data_sur['Age'] > 30]

In [ ]:
#畫出柱狀圖觀察
df = pd.DataFrame({'Age':['young', 'older'], 'val':[len(df_young), len(df_older)]})
ax = df.plot.bar(x='Age', y='val', rot=0)

In [ ]:
data

### 缺失值處理
在前面的章節應該有提過缺失值的處理，缺失值的存在會讓模型無法訓練，而我們因為資料很少，所以用補值的方式。

In [ ]:
# fillna(999)會讓所有缺失值都被補成999，之所以選999是因為不認為資料集的任何欄位中有999這個數字。
data = data.fillna(999)

In [ ]:
data

### 分割資料並訓練模型
接下來我們把前750筆資料當作訓練資料`X_train`，750筆之後的則當作測試資料。並且把`Survived`作為`y_train`從訓練資料中獨立出來。

我們使用決策樹 DecisionTreeClassifier() 來進行訓練。

In [ ]:
# 分割資料
X_train = data[:750]
X_test = data[750:]

# 把`Survived`作為`y_train`從訓練資料中獨立出來。
y_train = X_train.pop('Survived')

# 建立並訓練(fit)決策樹
clf = tree.DecisionTreeClassifier() 
clf = clf.fit(X_train, y_train)

### 評估模型正確率
最後我們想要知道模型的準確率到底有多好，這裡我們選擇了Accuracy和Recall兩個metric(指標)來評估
- Accuracy代表正確率，也就是模型每一次預測的準確度。
- Recall則是召回率，代表模型是否能把資料集中沒能獲救的人都找出來。

In [ ]:
from sklearn.metrics import accuracy_score, recall_score

# 從測試資料集取出y值作為真實答案
y_test = X_test.pop('Survived')

# 進行預測，取得預測答案
y_pred = clf.predict(X_test) 

print('accuracy_score=',accuracy_score(y_test, y_pred))
print('recall_score=',recall_score(y_test, y_pred))


In [ ]:
# 我們現在來用這個已經訓練好的model -> clf 來玩玩預測的結果
# 首先我們要再將一個用不到的欄位拿掉

data = data.drop(columns=['PassengerId'])

In [ ]:
data[:10]

In [ ]:
# 並且再fit 一次model
# 分割資料

X_train = data[:750]
X_test = data[750:]

# 把`Survived`作為`y_train`從訓練資料中獨立出來。

y_train = X_train.pop('Survived')

# 建立並訓練(fit)決策樹

clf = tree.DecisionTreeClassifier() 
clf = clf.fit(X_train, y_train)

我們來玩玩這個預測模型吧!


In [ ]:


# my_survival =[[Pclass, Sex, Age, Sibsp, Parch, Fare, Embarked]]

my_survival = [[2,0,20,3, 1,900, 0]]

pred = clf.predict(my_survival)

if pred == 0:
    print('Oh no! You did not survive')
else:
    print('Nice! You survived')